<a href="https://colab.research.google.com/github/gupta799/LLMFinetuning/blob/main/gkd_knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Distillation with GKDTrainer

This notebook demonstrates knowledge distillation using the **GKDTrainer** from TRL (Transformer Reinforcement Learning) library.

**Setup:**
- **Teacher Model:** Llama-3.1-8B-Instruct (8B parameters)
- **Student Model:** TinyLlama-1.1B-Chat-v1.0 (1.1B parameters)
- **Dataset:** argilla/distilabel-math-preference-dpo (math reasoning dataset)

**Goal:** Compress the knowledge from the 8B teacher model into the smaller 1.1B student model while maintaining strong mathematical reasoning capabilities.

**Note:** Both models use the Llama tokenizer architecture, ensuring vocabulary compatibility for GKD training.

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q transformers datasets trl peft accelerate bitsandbytes torch

In [ ]:
# Import libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from trl import GKDTrainer, GKDConfig
import os

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Load Models and Tokenizer

We'll load:
- **Teacher Model (Llama-3.1-8B-Instruct):** Using 4-bit quantization to fit in GPU memory
- **Student Model (TinyLlama-1.1B-Chat-v1.0):** Full precision for training
- **Tokenizer:** Shared Llama tokenizer (ensures vocabulary compatibility)

In [ ]:
# Model names
teacher_model_name = "meta-llama/Llama-3.1-8B-Instruct"
student_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer from teacher model (both models use Llama tokenizer)
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

# Set padding token to eos_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Tokenizer loaded: {len(tokenizer)} tokens in vocabulary")
print(f"Vocab size: {tokenizer.vocab_size}")

In [ ]:
# Configure 4-bit quantization for teacher model to save memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Loading teacher model (Llama-3.1-8B-Instruct with 4-bit quantization)...")
teacher_model = AutoModelForCausalLM.from_pretrained(
    teacher_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
teacher_model.eval()  # Set to evaluation mode

print(f"Teacher model loaded: {sum(p.numel() for p in teacher_model.parameters()) / 1e9:.2f}B parameters")
print(f"Teacher vocab size: {teacher_model.config.vocab_size}")

In [ ]:
print("Loading student model (TinyLlama-1.1B-Chat-v1.0)...")
student_model = AutoModelForCausalLM.from_pretrained(
    student_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
student_model.train()  # Set to training mode

print(f"Student model loaded: {sum(p.numel() for p in student_model.parameters()) / 1e9:.2f}B parameters")
print(f"Student vocab size: {student_model.config.vocab_size}")

# Verify vocabulary sizes match
print(f"\n✓ Vocabulary compatibility check:")
print(f"  Teacher vocab size: {teacher_model.config.vocab_size}")
print(f"  Student vocab size: {student_model.config.vocab_size}")
if teacher_model.config.vocab_size == student_model.config.vocab_size:
    print(f"  ✓ Vocabularies match! GKD training will work correctly.")
else:
    print(f"  ✗ WARNING: Vocabulary size mismatch detected!")

## 3. Dataset Preparation

We'll load the **argilla/distilabel-math-preference-dpo** dataset and format it for the GKDTrainer.

The GKDTrainer expects data in conversational format with role-content dictionaries.

In [ ]:
# Load dataset
print("Loading dataset...")
dataset = load_dataset("argilla/distilabel-math-preference-dpo", split="train")

# Take a subset for faster training (remove this line for full training)
dataset = dataset.select(range(min(5000, len(dataset))))

print(f"Dataset loaded: {len(dataset)} examples")
print(f"Dataset features: {dataset.features}")
print(f"\nSample example:")
print(dataset[0])

In [ ]:
# Format dataset for GKDTrainer
# The dataset should have conversational format with messages

def format_dataset(example):
    """Convert dataset to conversational format expected by GKDTrainer."""
    # Check if 'messages' field exists, otherwise create it
    if 'messages' not in example:
        # Construct messages from prompt and chosen response
        messages = [
            {"role": "user", "content": example.get('prompt', example.get('instruction', ''))},
            {"role": "assistant", "content": example.get('chosen', example.get('response', ''))}
        ]
        example['messages'] = messages
    
    return example

print("Formatting dataset...")
dataset = dataset.map(format_dataset)

# Split into train and eval
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"Train dataset: {len(train_dataset)} examples")
print(f"Eval dataset: {len(eval_dataset)} examples")

## 4. Configure GKDTrainer

**GKD Configuration Parameters:**
- **lmbda (λ):** Controls the proportion of student-generated outputs (0.5 = balanced)
- **beta (β):** Interpolates between forward KL and reverse KL divergence (0.5 = balanced)
- **seq_kd:** Whether to use sequence-level knowledge distillation (False = token-level)

In [ ]:
# Configure GKD training arguments
training_args = GKDConfig(
    output_dir="./gkd_llama_distillation",
    
    # GKD-specific parameters
    lmbda=0.5,  # Balance between supervised and on-policy learning
    beta=0.5,   # Balance between forward and reverse KL divergence
    seq_kd=False,  # Use token-level distillation
    
    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    
    # Optimization
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # Memory optimization
    gradient_checkpointing=True,
    bf16=True,
    
    # Logging and evaluation
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    
    # Other settings
    remove_unused_columns=False,
    report_to="none",  # Change to "wandb" if you want to log to Weights & Biases
)

print("GKD Configuration:")
print(f"  Lambda (λ): {training_args.lmbda}")
print(f"  Beta (β): {training_args.beta}")
print(f"  Sequence KD: {training_args.seq_kd}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

## 5. Initialize GKDTrainer and Start Training

In [ ]:
# Initialize GKDTrainer
print("Initializing GKDTrainer...")
trainer = GKDTrainer(
    model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("GKDTrainer initialized successfully!")
print(f"\nStarting knowledge distillation training...")

In [ ]:
# Start training
train_result = trainer.train()

print("\n=" * 50)
print("Training completed!")
print("=" * 50)
print(f"\nTraining metrics:")
print(f"  Final loss: {train_result.metrics['train_loss']:.4f}")
print(f"  Training time: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"  Samples per second: {train_result.metrics['train_samples_per_second']:.2f}")

In [ ]:
# Save the distilled student model
print("\nSaving distilled student model...")
output_dir = "./distilled_tinyllama_1.1b_math"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to: {output_dir}")

## 6. Evaluation and Testing

Let's test the distilled student model on sample math problems and compare with the teacher.

In [ ]:
# Test prompts
test_prompts = [
    "Solve the equation: 3x + 7 = 22",
    "What is the derivative of f(x) = x^3 + 2x^2 - 5x + 1?",
    "If a train travels at 60 mph for 2.5 hours, how far does it travel?",
    "Calculate the area of a circle with radius 5 cm.",
]

print("Testing the distilled student model on math problems...\n")
print("=" * 80)

In [ ]:
# Set models to eval mode
student_model.eval()
teacher_model.eval()

def generate_response(model, prompt, max_new_tokens=200):
    """Generate response from a model."""
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

# Test each prompt
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n📝 Test {i}: {prompt}")
    print("-" * 80)
    
    # Get student response
    print("\n🎓 Student Model (Distilled TinyLlama-1.1B):")
    student_response = generate_response(student_model, prompt)
    print(student_response)
    
    # Get teacher response
    print("\n👨‍🏫 Teacher Model (Llama-3.1-8B):")
    teacher_response = generate_response(teacher_model, prompt)
    print(teacher_response)
    
    print("\n" + "=" * 80)

## 7. Model Comparison Summary

Let's compare the models:

In [ ]:
# Calculate model sizes
def get_model_size(model):
    """Calculate model size in GB."""
    param_size = sum(p.nelement() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.nelement() * b.element_size() for b in model.buffers())
    return (param_size + buffer_size) / (1024**3)

teacher_size = get_model_size(teacher_model)
student_size = get_model_size(student_model)

print("\n" + "=" * 80)
print("Model Comparison Summary")
print("=" * 80)
print(f"\n📊 Teacher Model (Llama-3.1-8B-Instruct):")
print(f"   - Parameters: ~8B")
print(f"   - Memory footprint: {teacher_size:.2f} GB (4-bit quantized)")
print(f"\n📊 Student Model (Distilled TinyLlama-1.1B):")
print(f"   - Parameters: ~1.1B")
print(f"   - Memory footprint: {student_size:.2f} GB")
print(f"\n🎯 Compression Ratio: ~7.3x smaller")
print(f"   - Size reduction: {(1 - student_size/teacher_size) * 100:.1f}%")
print("\n✅ Benefits of the distilled student model:")
print("   - Much faster inference")
print("   - Lower memory requirements")
print("   - Suitable for edge deployment")
print("   - Retains mathematical reasoning capabilities from teacher")
print("   - Compatible with Llama ecosystem")

## Conclusion

In this notebook, we successfully used the **GKDTrainer** from TRL to distill knowledge from the **Llama-3.1-8B-Instruct** teacher model into the smaller **TinyLlama-1.1B-Chat-v1.0** student model.

**Key Takeaways:**
1. ✅ The GKDTrainer implements Generalized Knowledge Distillation, addressing train-inference distribution mismatch
2. ✅ Using `lmbda=0.5` balances supervised learning with on-policy learning
3. ✅ Using `beta=0.5` balances forward and reverse KL divergence
4. ✅ 4-bit quantization allows loading large teacher models on consumer GPUs
5. ✅ The distilled model is ~7x smaller while retaining strong math reasoning
6. ✅ Both models use the Llama tokenizer, ensuring vocabulary compatibility

**Why This Model Pair Works:**
- Both Llama-3.1-8B and TinyLlama-1.1B use the same tokenizer architecture
- Identical vocabulary sizes prevent tensor shape mismatches during training
- TinyLlama was specifically designed to be compatible with the Llama ecosystem

**Next Steps:**
- Fine-tune hyperparameters (lmbda, beta) for better performance
- Try different datasets for domain-specific distillation
- Experiment with larger training datasets
- Evaluate on standard benchmarks (GSM8K, MATH, etc.)
- Deploy the distilled model for production use